In [1]:
import pandas as pd
import numpy as np
import subprocess
import random
from scipy import stats
import glob
import math
import csv
import sys
import os

import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['font.family'] = "serif"
%matplotlib inline

In [3]:
# let's begin putting together the transcriptome simulation set

# What we need
# number of transcripts to simulate (what is an average number of transcripts per tissue?)
# number of loci (average for a tissue)
# for each locus need to draw the number of transcripts of each category, such that the final distributions
#     are close to tissue averages

# then for each transcript need to draw from the distribution of total contribution to expression
#     such that the final distributions resemble those observed in real data

In [2]:
# declarations
base_dir_data = "/home/sparrow/JHU/gtex_stats/data/"
base_dir_out = "/home/sparrow/JHU/gtex_stats/out_test/"
out_dir = "/home/sparrow/JHU/tx_noise/sim_samples_single_joint_rsem/"
hg38_fa = "/home/sparrow/genomicData/hg38/hg38_p8.fa"

genRNAseq = "/home/varabyou/genomicTools/genRNAseq_noFold_cov.R"

readlen = 101
num_samples = 2

gff3cols=["seqid","source","type","start","end","score","strand","phase","attributes"]
rsem_tpm_cols = ["transcript_id","gene_id","length","effective_length","expected_count","TPM","FPKM","IsoPct"]

In [3]:
stats = {
        "starting number of real and splicing noise loci":0,
        "starting number of splicing noise loci":0,
        "starting number of intronic noise loci":0,
        "starting number of polymerase noise loci": 0,
        "adjusted starting number of real only loci":0,
        "adjusted starting number of splicing noise loci":0,
        "adjusted starting number of intronic noise loci":0,
        "adjusted starting number of polymerase noise loci": 0,
        "average number of real loci":0,
        "average number of noise loci":0,
        "average number of undefined loci":0,
    }

In [4]:
# load base annotations
print(">>>loading base annotations")
real_baseDF = pd.read_csv(base_dir_data+"ALL.combined.IDs.olny.in.ALL.combined.true.gtf",sep="\t",names=gff3cols)
real_baseDF = real_baseDF[real_baseDF["type"]=="transcript"].reset_index(drop=True)
nonint_baseDF = pd.read_csv(base_dir_data+"ALL.combined.IDs.olny.in.ALL.combined.no.contained.non.intronic.gtf",sep="\t",names=gff3cols)
nonint_baseDF = nonint_baseDF[nonint_baseDF["type"]=="transcript"].reset_index(drop=True)
int_baseDF = pd.read_csv(base_dir_data+"ALL.combined.IDs.olny.in.ALL.combined.no.contained.intronic.gtf",sep="\t",names=gff3cols)
int_baseDF = int_baseDF[int_baseDF["type"]=="transcript"].reset_index(drop=True)
pol_baseDF = pd.read_csv(base_dir_data+"ALL.combined.IDs.olny.in.ALL.combined.no.contained.RNApol.gtf",sep="\t",names=gff3cols)
pol_baseDF = pol_baseDF[pol_baseDF["type"]=="transcript"].reset_index(drop=True)

>>>loading base annotations


In [5]:
# get all loci and transcript IDs
print(">>>getting loci IDs")
real_baseDF["lid"] = real_baseDF["attributes"].str.split("gene_id \"",expand=True)[1].str.split("\"",expand=True)[0]
nonint_baseDF["lid"] = nonint_baseDF["attributes"].str.split("gene_id \"",expand=True)[1].str.split("\"",expand=True)[0]
int_baseDF["lid"] = int_baseDF["attributes"].str.split("gene_id \"",expand=True)[1].str.split("\"",expand=True)[0]
pol_baseDF["lid"] = pol_baseDF["attributes"].str.split("gene_id \"",expand=True)[1].str.split("\"",expand=True)[0]
real_locs = set(real_baseDF["lid"])
nonint_locs = set(nonint_baseDF["lid"])
int_locs = set(int_baseDF["lid"])
pol_locs = set(pol_baseDF["lid"])
stats["starting number of real and splicing noise loci"] = len(real_locs)
stats["starting number of splicing noise loci"] = len(nonint_locs)
stats["starting number of intronic loci"] = len(int_locs)
stats["starting number of polymerase loci"] = len(pol_locs)

total_n_locs = len(real_locs)+len(nonint_locs)+len(int_locs)+len(pol_locs)

>>>getting loci IDs


In [6]:
# perform cleanup, by removing any loci in 
#   1. int that are not in real
#   2. nonint that are not in real
#   3. pol that are in real
int_locs = int_locs - int_locs.difference(real_locs)
assert(len(int_locs.difference(real_locs))==0),"something wrong intronic"
int_baseDF = int_baseDF[int_baseDF["lid"].isin(int_locs)].reset_index(drop=True)

nonint_locs = nonint_locs - nonint_locs.difference(real_locs)
assert(len(nonint_locs) == len(real_locs.intersection(nonint_locs))),"something wrong non-intronic"
nonint_baseDF = nonint_baseDF[nonint_baseDF["lid"].isin(nonint_locs)].reset_index(drop=True)

pol_locs = pol_locs - real_locs.intersection(pol_locs)
assert(len(real_locs.intersection(pol_locs))==0),"something wrong polymerase"
pol_baseDF = pol_baseDF[pol_baseDF["lid"].isin(pol_locs)].reset_index(drop=True)

real_baseDF["tid"] = real_baseDF["attributes"].str.split("transcript_id \"",expand=True)[1].str.split("\"",expand=True)[0]
real_baseDF = real_baseDF[["lid","tid"]]
nonint_baseDF["tid"] = nonint_baseDF["attributes"].str.split("transcript_id \"",expand=True)[1].str.split("\"",expand=True)[0]
nonint_baseDF = nonint_baseDF[["lid","tid"]]
int_baseDF["tid"] = int_baseDF["attributes"].str.split("transcript_id \"",expand=True)[1].str.split("\"",expand=True)[0]
int_baseDF = int_baseDF[["lid","tid"]]
pol_baseDF["tid"] = pol_baseDF["attributes"].str.split("transcript_id \"",expand=True)[1].str.split("\"",expand=True)[0]
pol_baseDF = pol_baseDF[["lid","tid"]]

In [7]:
# get the averages
avg_real_locs = 0
avg_noise_locs = 0
avg_undefined_locs = 0
with open(base_dir_out+"res_distrib.loc_stats","r") as inFP:
    for line in inFP.readlines():
        type_loc,num_loc = line.split(":")
        if type_loc=="real":
            avg_real_locs = int(num_loc)
        elif type_loc=="noise":
            avg_noise_locs = int(num_loc)
        elif type_loc=="undefined":
            if(int(num_loc))>0:
                avg_undefined_locs = int(num_loc)
        else:
            print("error in loading averages")
stats["average number of real loci"] = avg_real_locs
stats["average number of noise loci"] = avg_noise_locs
stats["average number of undefined loci"] = avg_undefined_locs

In [50]:
# randomly select loci of each type based on the averages
# noise loci are a combination of: intronic and polymerase
# this needs to be done proportionally to the consistent parts of each group (real(real,nonint) and noise(int,pol))

real_only_locs = real_locs - nonint_locs

# in case of inconsistencies associated with merging transcripts of different types under the same locus
# we need to scale the averages with respect to the new numbers
avg_total_locs = avg_real_locs+avg_noise_locs
sum_locs_to_choose_from = len(real_only_locs)+len(nonint_locs)+len(int_locs)+len(pol_locs)

stats["adjusted number of real only loci"] = len(real_only_locs)
stats["adjusted number of splicing noise loci"] = len(nonint_locs)
stats["adjusted number of intronic loci"] = len(int_locs)
stats["adjusted number of polymerase loci"] = len(pol_locs)

perc_real = len(real_only_locs)/len(real_locs)
perc_nonint = len(nonint_locs)/len(real_locs)
assert(perc_real+perc_nonint)==1,"wrong percent real and nonint"

perc_int = len(int_locs)/(len(int_locs)+len(pol_locs))
perc_pol = len(pol_locs)/(len(int_locs)+len(pol_locs))
assert((perc_int+perc_pol)==1),"wrong percent pol and int"
stats["number of selected real locs"] = int(avg_real_locs*perc_real)
stats["number of selected splicing noise locs"] = int(avg_real_locs*perc_nonint)
stats["number of selected intronic loci locs"] = int(avg_noise_locs*perc_int)
stats["number of selected polymerase locs"] = int(avg_noise_locs*perc_pol)

real_only_locs_rand = np.random.choice(list(real_only_locs),int(avg_real_locs*perc_real), replace=False)
nonint_locs_rand = np.random.choice(list(nonint_locs),int(avg_real_locs*perc_nonint), replace=False)
int_locs_rand = np.random.choice(list(int_locs),int(avg_noise_locs*perc_int), replace=False)
pol_locs_rand = np.random.choice(list(pol_locs),int(avg_noise_locs*perc_pol), replace=False)

In [51]:
real_only_baseDF = real_baseDF[real_baseDF["lid"].isin(real_only_locs_rand)].reset_index(drop=True)
real_nonint_baseDF = real_baseDF[(real_baseDF["lid"].isin(nonint_locs_rand))&~(real_baseDF["lid"].isin(real_only_locs_rand))].reset_index(drop=True)
nonint_baseDF = nonint_baseDF[nonint_baseDF["lid"].isin(nonint_locs_rand)].reset_index(drop=True)
int_baseDF = int_baseDF[int_baseDF["lid"].isin(int_locs_rand)].reset_index(drop=True)
pol_baseDF = pol_baseDF[pol_baseDF["lid"].isin(pol_locs_rand)].reset_index(drop=True)

In [52]:
# load the transcript tissue distribution
ttx_real = list()
ttx_nonint = list()
ttx_int = list()
ttx_pol = list()

tmp_tx = pd.read_csv(base_dir_out+"res_distrib.num_tx_per_tissue_loc2")
tmp_tx_real_loc = tmp_tx[(tmp_tx["real"]>0)][["total","real","nonintronic"]].reset_index(drop=True)
tmp_tx_real_loc["perc_real"] = tmp_tx_real_loc["real"]/tmp_tx_real_loc["total"]
tmp_tx_real_loc["perc_nonint"] = tmp_tx_real_loc["nonintronic"]/tmp_tx_real_loc["total"]
d_real = list(tmp_tx_real_loc["perc_real"])
d_nonint = list(tmp_tx_real_loc["perc_nonint"])

tmp_tx_int_loc = tmp_tx[(tmp_tx["intronic"]>0)][["total","intronic"]].reset_index(drop=True)
tmp_tx_int_loc["perc_int"] = tmp_tx_int_loc["intronic"]/tmp_tx_int_loc["total"]
d_int = list(tmp_tx_int_loc["perc_int"])

tmp_tx_pol_loc = tmp_tx[(tmp_tx["polymerase"]>0)][["total","polymerase"]].reset_index(drop=True)
tmp_tx_pol_loc["perc_pol"] = tmp_tx_pol_loc["polymerase"]/tmp_tx_pol_loc["total"]
d_pol = list(tmp_tx_pol_loc["perc_pol"])

In [53]:
# now that we have the loci, we can subset the gtfs and move on to working with the transcripts
# the question is: how do we allocate transcripts per locus?
# do we do this based on the distribution of the number of transcripts per each type of locus?
# are there any caveates in the co-dependencies such as between real and non-intronic?

# we should probably only select transcripts of each group within each type of locus dataframe

# one important consideration is that the total number of transcripts should be close to what is observed
#  in an average tissue (needs to be computed in the gtex_stats)
# this approach, however, might be overly complicated, and simply randomly selecting n transcripts for each group
#  based on distribution computed by gtex_stats might be sufficient, and should approximate well to desired values

real_only_txs = set()
for name, group in real_only_baseDF.groupby("lid"):
    tids = group["tid"].tolist()
    nt = math.ceil(d_real[random.randint(0,len(d_real)-1)]*len(tids))
    random.shuffle(tids)
    real_only_txs.update(tids[:nt])

real_nonint_txs = set()
for name, group in real_nonint_baseDF.groupby("lid"):
    tids = group["tid"].tolist()
    nt = math.ceil(d_real[random.randint(0,len(d_real)-1)]*len(tids))
    random.shuffle(tids)
    real_nonint_txs.update(tids[:nt])

nonint_txs = set()
for name, group in nonint_baseDF.groupby("lid"):
    tids = group["tid"].tolist()
    nt = math.ceil(d_nonint[random.randint(0,len(d_nonint)-1)]*len(tids))
    random.shuffle(tids)
    nonint_txs.update(tids[:nt])

int_txs = set()
for name, group in int_baseDF.groupby("lid"):
    tids = group["tid"].tolist()
    nt = math.ceil(d_int[random.randint(0,len(d_int)-1)]*len(tids))
    random.shuffle(tids)
    int_txs.update(tids[:nt])

pol_txs = set()
for name, group in pol_baseDF.groupby("lid"):
    tids = group["tid"].tolist()
    nt = math.ceil(d_pol[random.randint(0,len(d_pol)-1)]*len(tids))
    random.shuffle(tids)
    pol_txs.update(tids[:nt])

stats["number of real only txs"] = len(real_only_txs)
stats["number of real nonint txs"] = len(real_nonint_txs)
stats["number of nonint txs"] = len(nonint_txs)
stats["number of int txs"] = len(int_txs)
stats["number of pol txs"] = len(pol_txs)

In [54]:
# now that we have lists of transcripts - we can subset the annotation further
print("number of real only txs before: "+str(len(real_only_baseDF)))
real_only_baseDF = real_only_baseDF[real_only_baseDF["tid"].isin(real_only_txs)].reset_index(drop=True)
print("number of real nonint txs before: "+str(len(real_nonint_baseDF)))
real_nonint_baseDF = real_nonint_baseDF[real_nonint_baseDF["tid"].isin(real_nonint_txs)].reset_index(drop=True)
print("number of nonint txs before: "+str(len(nonint_baseDF)))
nonint_baseDF = nonint_baseDF[nonint_baseDF["tid"].isin(nonint_txs)].reset_index(drop=True)
print("number of int txs before: "+str(len(int_baseDF)))
int_baseDF = int_baseDF[int_baseDF["tid"].isin(int_txs)].reset_index(drop=True)
print("number of pol txs before: "+str(len(pol_baseDF)))
pol_baseDF = pol_baseDF[pol_baseDF["tid"].isin(pol_txs)].reset_index(drop=True)

number of real only txs before: 11567
number of real nonint txs before: 26797
number of nonint txs before: 105942
number of int txs before: 8676
number of pol txs before: 36360


In [55]:
# lastly we need to generate samples based on the underlying annotation
# for this we will need additional information, similar to the tissue_per_loc2 in order to tell
# how likely a sample is to contribute a transcript to a tissue

# load the transcript sample distribution
ttx_real = list()
ttx_nonint = list()
ttx_int = list()
ttx_pol = list()

tmp_tx = pd.read_csv(base_dir_out+"res_distrib.num_tx_per_sample_loc2")
tmp_tx_real_loc = tmp_tx[(tmp_tx["real"]>0)][["total","real","nonintronic"]].reset_index(drop=True)
tmp_tx_real_loc["perc_real"] = tmp_tx_real_loc["real"]/tmp_tx_real_loc["total"]
tmp_tx_real_loc["perc_nonint"] = tmp_tx_real_loc["nonintronic"]/tmp_tx_real_loc["total"]
d_real = list(tmp_tx_real_loc["perc_real"])
d_nonint = list(tmp_tx_real_loc["perc_nonint"])

tmp_tx_int_loc = tmp_tx[(tmp_tx["intronic"]>0)][["total","intronic"]].reset_index(drop=True)
tmp_tx_int_loc["perc_int"] = tmp_tx_int_loc["intronic"]/tmp_tx_int_loc["total"]
d_int = list(tmp_tx_int_loc["perc_int"])

tmp_tx_pol_loc = tmp_tx[(tmp_tx["polymerase"]>0)][["total","polymerase"]].reset_index(drop=True)
tmp_tx_pol_loc["perc_pol"] = tmp_tx_pol_loc["polymerase"]/tmp_tx_pol_loc["total"]
d_pol = list(tmp_tx_pol_loc["perc_pol"])

In [56]:
real_only_txs = [set() for x in range(num_samples)]
for name, group in real_only_baseDF.groupby("lid"):
    tids = group["tid"].tolist()
    for i in range(num_samples):
        nt = math.ceil(d_real[random.randint(0,len(d_real)-1)]*len(tids))
        random.shuffle(tids)
        real_only_txs[i].update(tids[:nt])

real_nonint_txs = [set() for x in range(num_samples)]
for name, group in real_nonint_baseDF.groupby("lid"):
    tids = group["tid"].tolist()
    for i in range(num_samples):
        nt = math.ceil(d_real[random.randint(0,len(d_real)-1)]*len(tids))
        random.shuffle(tids)
        real_nonint_txs[i].update(tids[:nt])

nonint_txs = [set() for x in range(num_samples)]
for name, group in nonint_baseDF.groupby("lid"):
    tids = group["tid"].tolist()
    for i in range(num_samples):
        nt = math.ceil(d_nonint[random.randint(0,len(d_nonint)-1)]*len(tids))
        random.shuffle(tids)
        nonint_txs[i].update(tids[:nt])

int_txs = [set() for x in range(num_samples)]
for name, group in int_baseDF.groupby("lid"):
    tids = group["tid"].tolist()
    for i in range(num_samples):
        nt = math.ceil(d_int[random.randint(0,len(d_int)-1)]*len(tids))
        random.shuffle(tids)
        int_txs[i].update(tids[:nt])

pol_txs = [set() for x in range(num_samples)]
for name, group in pol_baseDF.groupby("lid"):
    tids = group["tid"].tolist()
    for i in range(num_samples):
        nt = math.ceil(d_pol[random.randint(0,len(d_pol)-1)]*len(tids))
        random.shuffle(tids)
        pol_txs[i].update(tids[:nt])

In [57]:
# the last thing we need to do is to guarantee that no expressions are smaller than 1
# the best way to do this is
# get the minimum of all coverages

In [58]:
# now that we have sample specific stuff - we can generate sample-specific GTFs
int_baseDF = pd.read_csv(base_dir_data+"ALL.combined.IDs.olny.in.ALL.combined.no.contained.intronic.gtf",sep="\t",names=gff3cols)
pol_baseDF = pd.read_csv(base_dir_data+"ALL.combined.IDs.olny.in.ALL.combined.no.contained.RNApol.gtf",sep="\t",names=gff3cols)
int_baseDF["tid"] = int_baseDF["attributes"].str.split("transcript_id \"",expand=True)[1].str.split("\"",expand=True)[0]
pol_baseDF["tid"] = pol_baseDF["attributes"].str.split("transcript_id \"",expand=True)[1].str.split("\"",expand=True)[0]

for i in range(num_samples):
    int_baseDF[int_baseDF["tid"].isin(int_txs[i])][gff3cols].to_csv(out_dir+"/res_distrib.int.sample"+str(i)+".gtf",sep="\t",index=False,header=False,quoting=csv.QUOTE_NONE)
    pol_baseDF[pol_baseDF["tid"].isin(pol_txs[i])][gff3cols].to_csv(out_dir+"/res_distrib.pol.sample"+str(i)+".gtf",sep="\t",index=False,header=False,quoting=csv.QUOTE_NONE)

# load distribution of the sum of expressions
tmp_exp_int = pd.read_csv(base_dir_out+"res_distrib.cov_sample_intronic")
d_exp_int = list(tmp_exp_int["cov"]+1)

tmp_exp_pol = pd.read_csv(base_dir_out+"res_distrib.cov_sample_polymerase")
d_exp_pol = list(tmp_exp_pol["cov"]+1)

In [90]:
# now generate corresponding expression matrices for the transcripts in each sample for each type
for i in range(num_samples):        
    intDF = pd.read_csv(out_dir+"/res_distrib.int.sample"+str(i)+".gtf",sep="\t",names=gff3cols)
    nt_int = len(intDF[intDF["type"]=="transcript"])
    exps_int = np.random.choice(d_exp_int, nt_int, replace=False)
    np.savetxt(out_dir+"/res_distrib.int.sample"+str(i)+".exp", exps_int, delimiter=',',fmt='%0.2f')

    polDF = pd.read_csv(out_dir+"/res_distrib.pol.sample"+str(i)+".gtf",sep="\t",names=gff3cols)
    nt_pol = len(polDF[polDF["type"]=="transcript"])
    exps_pol = np.random.choice(d_exp_pol, nt_pol, replace=False)
    np.savetxt(out_dir+"/res_distrib.pol.sample"+str(i)+".exp", exps_pol, delimiter=',',fmt='%0.2f')

    # need to generate data for the RSEM simulation
    tdf = int_baseDF[int_baseDF["tid"].isin(int_txs[i])]
    edf = tdf[tdf["type"]=="exon"].reset_index(drop=True)
    tdf = tdf[tdf["type"]=="transcript"][["tid","start","end"]].reset_index(drop=True) # intended for order
    tdf["len"] = tdf["end"]-tdf["start"]
    tdf.drop(["start","end"],axis=1,inplace=True)
    edf["elen"] = edf["end"]-edf["start"]
    edf = edf[["tid","elen"]]
    edf = edf.groupby("tid").agg({"elen":"sum"}).reset_index()
    # now organize by
    tdf = tdf.merge(edf,on="tid",how="left",indicator=True)
    assert len(tdf) == len(tdf[tdf["_merge"]=="both"]),"missing/extra exons"

    intDF = pd.read_csv(out_dir+"/res_distrib.int.sample"+str(i)+".gtf",sep="\t",names=gff3cols)
    nt_int = len(intDF[intDF["type"]=="transcript"])
    exps_int = np.random.choice(d_exp_int, nt_int, replace=False)
    tdf["cov"] = exps_int
    tdf["cov"] = tdf["cov"].fillna(0)
    assert len(tdf[tdf["cov"]==0])==0,"missing coverages"
    tdf["nr"] = tdf["elen"]*tdf["cov"]
    tdf["kelen"] = tdf["elen"]/1000
    tdf["rpk"] = tdf["nr"]/tdf["kelen"]
    pm_sf = tdf["rpk"].sum()/1000000
    tdf["tpm"] = tdf["rpk"]/pm_sf
    tdf.drop(["_merge","cov","nr","kelen","rpk"],axis=1,inplace=True)
    tdf.columns = ["transcript_id","length","effective_length","TPM"]
    tdf["gene_id"] = tdf["transcript_id"]
    tdf["expected_count"] = 0.00
    tdf["FPKM"] = 0.00
    tdf["IsoPct"] = 0.00
    tdf[rsem_tpm_cols].to_csv(out_dir+"/res_distrib.int.sample"+str(i)+".rsem_tpms",sep="\t",index=False)

    # need to generate data for the RSEM simulation
    tdf = pol_baseDF[pol_baseDF["tid"].isin(pol_txs[i])]
    edf = tdf[tdf["type"]=="exon"].reset_index(drop=True)
    tdf = tdf[tdf["type"]=="transcript"][["tid","start","end"]].reset_index(drop=True) # intended for order
    tdf["len"] = tdf["end"]-tdf["start"]
    tdf.drop(["start","end"],axis=1,inplace=True)
    edf["elen"] = edf["end"]-edf["start"]
    edf = edf[["tid","elen"]]
    edf = edf.groupby("tid").agg({"elen":"sum"}).reset_index()
    # now organize by
    tdf = tdf.merge(edf,on="tid",how="left",indicator=True)
    assert len(tdf) == len(tdf[tdf["_merge"]=="both"]),"missing/extra exons"

    polDF = pd.read_csv(out_dir+"/res_distrib.pol.sample"+str(i)+".gtf",sep="\t",names=gff3cols)
    nt_pol = len(polDF[polDF["type"]=="transcript"])
    exps_pol = np.random.choice(d_exp_pol, nt_pol, replace=False)
    tdf["cov"] = exps_pol
    tdf["cov"] = tdf["cov"].fillna(0)
    assert len(tdf[tdf["cov"]==0])==0,"missing coverages"
    tdf["nr"] = tdf["elen"]*tdf["cov"]
    tdf["kelen"] = tdf["elen"]/1000
    tdf["rpk"] = tdf["nr"]/tdf["kelen"]
    pm_sf = tdf["rpk"].sum()/1000000
    tdf["tpm"] = tdf["rpk"]/pm_sf
    tdf.drop(["_merge","cov","nr","kelen","rpk"],axis=1,inplace=True)
    tdf.columns = ["transcript_id","length","effective_length","TPM"]
    tdf["gene_id"] = tdf["transcript_id"]
    tdf["expected_count"] = 0.00
    tdf["FPKM"] = 0.00
    tdf["IsoPct"] = 0.00
    tdf[rsem_tpm_cols].to_csv(out_dir+"/res_distrib.pol.sample"+str(i)+".rsem_tpms",sep="\t",index=False)

In [91]:
# now need to somehow get the tpms for the samples
# while intronic and polymerase are easy to sample
# real and nonintronic are correlated
# so we need to use the special distribution to sample

# first need to be able to process real and nonintronic together
# and output expression accordingly

# load joint expressions
real_baseDF = pd.read_csv(base_dir_data+"ALL.combined.IDs.olny.in.ALL.combined.true.gtf",sep="\t",names=gff3cols)
nonint_baseDF = pd.read_csv(base_dir_data+"ALL.combined.IDs.olny.in.ALL.combined.no.contained.non.intronic.gtf",sep="\t",names=gff3cols)
real_baseDF["tid"] = real_baseDF["attributes"].str.split("transcript_id \"",expand=True)[1].str.split("\"",expand=True)[0]
nonint_baseDF["tid"] = nonint_baseDF["attributes"].str.split("transcript_id \"",expand=True)[1].str.split("\"",expand=True)[0]
# load joint expressions
joined_df = pd.read_csv(base_dir_out+"res_distrib.tpms_joined")
joined_df["code"] = joined_df["nt_real"].astype(int).astype(str)+"-"+joined_df["nt_nonint"].astype(int).astype(str)
joined_df.drop(["nt_real","nt_nonint"],axis=1,inplace=True)

In [ ]:
for i in range(num_samples):
    sub_real = real_baseDF[real_baseDF["tid"].isin(real_only_txs[i].union(real_nonint_txs[i]))]
    sub_real["gid"] = sub_real["attributes"].str.split("gene_id \"",expand=True)[1].str.split("\"",expand=True)[0]
    sub_nonint = nonint_baseDF[nonint_baseDF["tid"].isin(nonint_txs[i])]
    sub_nonint["gid"] = sub_nonint["attributes"].str.split("gene_id \"",expand=True)[1].str.split("\"",expand=True)[0]

    gsr = sub_real[sub_real["type"]=="transcript"][["gid","tid"]].groupby("gid").count().reset_index()
    gsr.columns = ["gid","ntr"]
    gsn = sub_nonint[sub_nonint["type"]=="transcript"][["gid","tid"]].groupby("gid").count().reset_index()
    gsn.columns = ["gid","ntn"]
    gs = gsr.merge(gsn,on="gid",how="outer",indicator=True)
    print("removing "+str(len(gs[gs["_merge"]=="right_only"]))+" number of loci due to presence of nonint only transcripts")
    gs = gs[~(gs["_merge"]=="right_only")].reset_index(drop=True)
    gs["ntn"] = gs["ntn"].fillna(0)
    gs["ntr"] = gs["ntr"].fillna(0)
    gs["code"] = gs["ntr"].astype(int).astype(str)+"-"+gs["ntn"].astype(int).astype(str)
    gs.sort_values(by="code",inplace=True)

    real_exp = open(out_dir+"/res_distrib.real.sample"+str(i)+".exp","w+")
    nonint_exp = open(out_dir+"/res_distrib.nonint.sample"+str(i)+".exp","w+")

    jd = joined_df.copy(deep=True)
    # compute the number of each code in gs
    tmp = gs.groupby(by = 'code')['gid'].count().reset_index()
    tmp.columns = ["code","nc"]
    jd = jd.merge(tmp,on="code",how="inner",indicator=True)
    # need to have a way of knowing which loci have no code matches...
    # and to maintain the order between the expressions and corresponding gtfs
    jd = jd[jd["_merge"]=="both"]
    gs = gs[gs["code"].isin(set(jd["code"]))]
    # now we can select th
    jd = jd.groupby('code', as_index=False).apply(lambda obj: obj.loc[np.random.choice(obj.index, obj.nc.iloc[0],replace=True),:]).reset_index(drop=True)

    gs.sort_values(by="code",inplace=True,ascending=False)
    gs.reset_index(drop=True,inplace=True)
    jd.sort_values(by="code",inplace=True,ascending=False)
    jd.reset_index(drop=True,inplace=True)
    gs[["real","nonint","code_jd"]] = jd[["real","nonint","code"]]
    assert len(gs)==len(gs[gs["code"]==gs["code_jd"]]),"codes do not match"

    # now we can split the expressions
    rgs = gs[gs["ntr"]>0][["gid","real"]].reset_index(drop=True)
    rgs["reals"] = rgs["real"].str.split(";")
    rgs = rgs.explode("reals")
    rgs.drop("real",axis=1,inplace=True)
    # we can save the expressions, however, now we need to make sure the gtf
    # is saved in the same order
    sub_real = sub_real[sub_real["gid"].isin(rgs["gid"])]
    assert len(set(rgs["gid"]).intersection(set(sub_real["gid"])))==len(set(sub_real["gid"])),"incompatible gids in sub_real and gs"
    # to finally ensure the same order between the two dataframes we can
    # get the order of genes in one dataframe
    # and then merge into it independently both expression and gtf data
    #   to guarantee identical ordering
    gene_order = rgs[["gid"]]
    gene_order.drop_duplicates(keep="first",inplace=True)
    gene_order.reset_index(drop=True)
    real_gtf = gene_order.merge(sub_real,on="gid",how="inner")
    real_exp = gene_order.merge(rgs,on="gid",how="inner")
    real_exp["reals"] = real_exp["reals"].astype(float).round(2)+1.0
    real_gtf[gff3cols].to_csv(out_dir+"/res_distrib.real.sample"+str(i)+".gtf",sep="\t",index=False,header=False,quoting=csv.QUOTE_NONE)
    real_exp[["reals"]].to_csv(out_dir+"/res_distrib.real.sample"+str(i)+".exp",index=False,header=False)
    
    # now we can split the expressions for the nonint
    ngs = gs[gs["ntn"]>0][["gid","nonint"]].reset_index(drop=True)
    ngs["nonints"] = ngs["nonint"].str.split(";")
    ngs = ngs.explode("nonints")
    ngs.drop("nonint",axis=1,inplace=True)
    # we can save the expressions, however, now we need to make sure the gtf
    # is saved in the same order
    sub_nonint = sub_nonint[sub_nonint["gid"].isin(ngs["gid"])]
    assert len(set(ngs["gid"]).intersection(set(sub_nonint["gid"])))==len(set(sub_nonint["gid"])),"incompatible gids in sub_nonint and gs"
    # to finally ensure the same order between the two dataframes we can
    # get the order of genes in one dataframe
    # and then merge into it independently both expression and gtf data
    #   to guarantee identical ordering
    gene_order = ngs[["gid"]]
    gene_order.drop_duplicates(keep="first",inplace=True)
    gene_order.reset_index(drop=True)
    nonint_gtf = gene_order.merge(sub_nonint,on="gid",how="inner")
    nonint_exp = gene_order.merge(ngs,on="gid",how="inner")
    nonint_exp["nonints"] = nonint_exp["nonints"].astype(float).round(2)+1.0
    nonint_gtf[gff3cols].to_csv(out_dir+"/res_distrib.nonint.sample"+str(i)+".gtf",sep="\t",index=False,header=False,quoting=csv.QUOTE_NONE)
    nonint_exp[["nonints"]].to_csv(out_dir+"/res_distrib.nonint.sample"+str(i)+".exp",index=False,header=False)
    
    # process for RSEM
    edf = real_gtf[real_gtf["type"]=="exon"].reset_index(drop=True)
    tdf = real_gtf[real_gtf["type"]=="transcript"][["tid","start","end"]].reset_index(drop=True) # intended for order
    tdf["len"] = tdf["end"]-tdf["start"]
    tdf.drop(["start","end"],axis=1,inplace=True)
    edf["elen"] = edf["end"]-edf["start"]
    edf = edf[["tid","elen"]]
    edf = edf.groupby("tid").agg({"elen":"sum"}).reset_index()
    # now organize by
    tdf = tdf.merge(edf,on="tid",how="left",indicator=True)
    assert len(tdf) == len(tdf[tdf["_merge"]=="both"]),"missing/extra exons"

    tdf["cov"] = real_exp["reals"]
    tdf["cov"] = tdf["cov"].fillna(0)
    assert len(tdf[tdf["cov"]==0])==0,"missing coverages"
    tdf["nr"] = tdf["elen"]*tdf["cov"]
    tdf["kelen"] = tdf["elen"]/1000
    tdf["rpk"] = tdf["nr"]/tdf["kelen"]
    pm_sf = tdf["rpk"].sum()/1000000
    tdf["tpm"] = tdf["rpk"]/pm_sf
    tdf.drop(["_merge","cov","nr","kelen","rpk"],axis=1,inplace=True)
    tdf.columns = ["transcript_id","length","effective_length","TPM"]
    tdf["gene_id"] = tdf["transcript_id"]
    tdf["expected_count"] = 0.00
    tdf["FPKM"] = 0.00
    tdf["IsoPct"] = 0.00
    tdf[rsem_tpm_cols].to_csv(out_dir+"/res_distrib.real.sample"+str(i)+".rsem_tpms",sep="\t",index=False)

    edf = nonint_gtf[nonint_gtf["type"]=="exon"].reset_index(drop=True)
    tdf = nonint_gtf[nonint_gtf["type"]=="transcript"][["tid","start","end"]].reset_index(drop=True) # intended for order
    tdf["len"] = tdf["end"]-tdf["start"]
    tdf.drop(["start","end"],axis=1,inplace=True)
    edf["elen"] = edf["end"]-edf["start"]
    edf = edf[["tid","elen"]]
    edf = edf.groupby("tid").agg({"elen":"sum"}).reset_index()
    # now organize by
    tdf = tdf.merge(edf,on="tid",how="left",indicator=True)
    assert len(tdf) == len(tdf[tdf["_merge"]=="both"]),"missing/extra exons"

    tdf["cov"] = nonint_exp["nonints"]
    tdf["cov"] = tdf["cov"].fillna(0)
    assert len(tdf[tdf["cov"]==0])==0,"missing coverages"
    tdf["nr"] = tdf["elen"]*tdf["cov"]
    tdf["kelen"] = tdf["elen"]/1000
    tdf["rpk"] = tdf["nr"]/tdf["kelen"]
    pm_sf = tdf["rpk"].sum()/1000000
    tdf["tpm"] = tdf["rpk"]/pm_sf
    tdf.drop(["_merge","cov","nr","kelen","rpk"],axis=1,inplace=True)
    tdf.columns = ["transcript_id","length","effective_length","TPM"]
    tdf["gene_id"] = tdf["transcript_id"]
    tdf["expected_count"] = 0.00
    tdf["FPKM"] = 0.00
    tdf["IsoPct"] = 0.00
    tdf[rsem_tpm_cols].to_csv(out_dir+"/res_distrib.nonint.sample"+str(i)+".rsem_tpms",sep="\t",index=False)

In [131]:
l = [1,2,3,4]
pd.DataFrame(l)

,0
0,1
1,2
2,3
3,4


In [130]:
exp_real = pd.read_csv(out_dir+"/res_distrib.real.sample0.rsem_tpms.tmp",sep="\t")
exp_real

,transcript_id,gene_id,length,effective_length,expected_count,TPM,FPKM,IsoPct
0,ALL_00201782,ALL_00201782,2444,1130,0.0,7.904005,0.0,0.0
1,ALL_00201784,ALL_00201784,1418,1418,0.0,2.261712,0.0,0.0
2,ALL_00201785,ALL_00201785,1373,1024,0.0,55.184178,0.0,0.0
3,ALL_00201786,ALL_00201786,2003,1224,0.0,236.832432,0.0,0.0
4,ALL_00201787,ALL_00201787,5573,1987,0.0,10.765111,0.0,0.0
...,...,...,...,...,...,...,...,...
19231,ALL_00339167,ALL_00339167,500130,1157,0.0,54.169205,0.0,0.0
19232,ALL_00330244,ALL_00330244,103254,2308,0.0,0.927062,0.0,0.0
19233,ALL_00330257,ALL_00330257,41718,902,0.0,447.635194,0.0,0.0
19234,ALL_00330260,ALL_00330260,4766,2887,0.0,174.279707,0.0,0.0
